# milkbasket Recommendation System for Purchase Data



In [55]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
import sys
sys.path.append("..")
import scripts.data_layer as data_layer
print "hello"

hello


In [56]:
customers = pd.read_csv('../data/recommend_1.csv')
transactions = pd.read_csv('../data/trx_data.csv')

In [57]:
print(customers.shape)
customers.head()

(436, 5)


,customerId,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,2698080,NaN,NaN,NaN,NaN
1,1134224,NaN,NaN,NaN,NaN
2,3686704,NaN,NaN,NaN,NaN
3,4045328,NaN,NaN,NaN,NaN
4,3568656,NaN,NaN,NaN,NaN


In [58]:
print(transactions.shape)
transactions.head()

(436, 5)


,customerId,products,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,2698080,1120336|1681904|1676304|2207184|1120336|11203...,NaN,NaN,NaN
1,1134224,1939280|1570912|1570912|1673728|1939280|15709...,NaN,NaN,NaN
2,3686704,1681456|1683024|1666448|1673728|2377088|1667008,NaN,NaN,NaN
3,4045328,1120224|1120336|1872976|1120336|1120224|19088...,NaN,NaN,NaN
4,3568656,1120112|1647632|1694560|1902656|2045568|11201...,NaN,NaN,NaN


In [59]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,...,13,14,15,16,17,18,19,20,21,22
0,2698080,1120336.0,1681904.0,1676304.0,2207184.0,1120336.0,1120336.0,1728608.0,1122576.0,1652672.0,...,2165968.0,1684928.0,1687392.0,1677536.0,1668240.0,2207184.0,1596784.0,1120336.0,1120224.0,1120336.0
1,1134224,1939280.0,1570912.0,1570912.0,1673728.0,1939280.0,1570912.0,1570912.0,1673728.0,1570912.0,...,1570912.0,1939280.0,1939280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,1134224,1570912.0,8
1,1134224,1673728.0,2
2,1134224,1939280.0,6
3,2698080,1120112.0,1
4,2698080,1120224.0,1
5,2698080,1120336.0,5
6,2698080,1122576.0,1
7,2698080,1596784.0,1
8,2698080,1652672.0,1
9,2698080,1668240.0,1


In [61]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

('Execution time:', 0.0, 'minutes')


In [62]:
print(data.shape)
data.head()

(3010, 3)


,customerId,productId,purchase_count
0,1121904,1120224,5
1,1121904,1120896,2
2,1121904,1682800,1
3,1121904,1688960,1
4,1121904,1694112,1


In [63]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [64]:
data_dummy = create_data_dummy(data)

In [65]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,1120112,1120224,1120336,1120448,1120560,1120672,1120784,1120896,1121008,1121120,...,2376864,2377088,2377200,2377312,2377424,2377536,2377648,2378432,2383360,2386496
customerId,,,,,,,,,,,,,,,,,,,,,
1121904,NaN,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1122128,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1122352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1133888,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1134224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
(df_matrix.shape)

(436, 746)

In [67]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(436, 746)


productId,1120112,1120224,1120336,1120448,1120560,1120672,1120784,1120896,1121008,1121120,...,2376864,2377088,2377200,2377312,2377424,2377536,2377648,2378432,2383360,2386496
customerId,,,,,,,,,,,,,,,,,,,,,
1121904,NaN,0.444444,NaN,NaN,NaN,NaN,NaN,0.333333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1122128,NaN,NaN,0.777778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1122352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1133888,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1134224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(1847, 3)


,customerId,productId,scaled_purchase_freq
3,1133888,1120112,1.000000
7,1153824,1120112,0.000000
11,1174880,1120112,0.777778
14,1184176,1120112,0.000000
15,1185744,1120112,0.000000


#### Define a function for normalizing data

In [69]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* We can normalize the their purchase history, from 0-1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

In [70]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

((2408, 3), (602, 3))


In [71]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [72]:
train_data

customerId,productId,purchase_count
4282320,1120224,4
1639792,1120112,4
1354080,1685824,1
1944208,1902656,2
1153824,1432928,1
4722816,2258144,1
4251184,2119152,1
2548000,2332960,1
4723040,1120448,1
4469696,1120672,8


In [73]:
test_data

customerId,productId,purchase_count
3019296,1122576,1
4616528,1683024,1
2520000,1693776,1
2485504,2043664,2
3843840,1120112,2
1475488,1120112,1
1622320,2356928,1
3039904,1685040,1
3397632,1683024,1
4544736,2035264,1


In [74]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [75]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [76]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [77]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 2408 observations with 421 users and 668 items.

Data prepared in: 0.005627s

2408 observations to process; with 668 unique items.

In [78]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

+------------+-----------+---------------+------+
| customerId | productId |     score     | rank |
+------------+-----------+---------------+------+
|  2698080   |  1144528  |      10.0     |  1   |
|  2698080   |  1120672  | 6.66666666667 |  2   |
|  2698080   |  1570912  |      5.0      |  3   |
|  2698080   |  2042096  |      4.0      |  4   |
|  2698080   |  1556464  |      4.0      |  5   |
|  2698080   |  1901872  |      4.0      |  6   |
|  2698080   |  2026640  | 3.83333333333 |  7   |
|  2698080   |  1120560  | 3.44117647059 |  8   |
|  2698080   |  1707888  | 3.39130434783 |  9   |
|  2698080   |  1120448  | 3.11111111111 |  10  |
|  1134224   |  1144528  |      10.0     |  1   |
|  1134224   |  1120672  | 6.66666666667 |  2   |
|  1134224   |  1120224  |   5.0390625   |  3   |
|  1134224   |  1570912  |      5.0      |  4   |
|  1134224   |  1120112  | 4.11904761905 |  5   |
|  1134224   |  2042096  |      4.0      |  6   |
|  1134224   |  1556464  |      4.0      |  7   |


#### Define a `model` function for model selection

In [79]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [80]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Using purchase dummy

In [81]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 2408 observations with 423 users and 654 items.

Data prepared in: 0.006577s

2408 observations to process; with 654 unique items.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|  2698080   |  1686160  |  1.0  |  1   |
|  2698080   |  1687504  |  1.0  |  2   |
|  2698080   |  1688064  |  1.0  |  3   |
|  2698080   |  1939280  |  1.0  |  4   |
|  2698080   |  1399216  |  1.0  |  5   |
|  2698080   |  1379840  |  1.0  |  6   |
|  2698080   |  1120560  |  1.0  |  7   |
|  2698080   |  1685040  |  1.0  |  8   |
|  2698080   |  1664320  |  1.0  |  9   |
|  2698080   |  1687952  |  1.0  |  10  |
|  1134224   |  1686160  |  1.0  |  1   |
|  1134224   |  1687504  |  1.0  |  2   |
|  1134224   |  1688064  |  1.0  |  3   |
|  1134224   |  1120112  |  1.0  |  4   |
|  1134224   |  1399216  |  1.0  |  5   |
|  1134224   |  1379840  |  1.0  |  6   |
|  1134224   |  1120560  |  1.0  |  7   |
|  1134224   |  1685040  |  1.0  |  8   |
|  1134224   |  1664320  |  1.0  |  9   |
|  1134224   |  1687952  |  1.0  |  10  |
|  3686704   |  1687504  |  1.0  |

#### Using normalized purchase count

In [82]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 1477 observations with 403 users and 118 items.

Data prepared in: 0.005017s

1477 observations to process; with 118 unique items.

+------------+-----------+----------------+------+
| customerId | productId |     score      | rank |
+------------+-----------+----------------+------+
|  2698080   |  1698032  |      1.0       |  1   |
|  2698080   |  1570912  | 0.535353535354 |  2   |
|  2698080   |  1120672  | 0.513888888889 |  3   |
|  2698080   |  1649872  |      0.5       |  4   |
|  2698080   |  1132096  |      0.5       |  5   |
|  2698080   |  2291968  |      0.5       |  6   |
|  2698080   |  1398656  |      0.5       |  7   |
|  2698080   |  1902320  |      0.5       |  8   |
|  2698080   |  2356928  |      0.5       |  9   |
|  2698080   |  1125600  |      0.5       |  10  |
|  1134224   |  1698032  |      1.0       |  1   |
|  1134224   |  1120672  | 0.513888888889 |  2   |
|  1134224   |  1121008  |      0.5       |  3   |
|  1134224   |  1649872  |      0.5       |  4   |
|  1134224   |  1132096  |      0.5       |  5   |
|  1134224   |  2291968  |      0.5       |  6   |
|  1134224   |  1398656  |     

In [83]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
1144528    10.000000
1120672     6.666667
1120224     5.039062
1570912     5.000000
1120112     4.119048
1901872     4.000000
2042096     4.000000
1556464     4.000000
2026640     3.833333
1120336     3.829268
1120560     3.441176
1707888     3.391304
1120448     3.111111
1571024     3.000000
1568672     3.000000
1690080     3.000000
1939280     2.357143
1966160     2.200000
2166080     2.153846
2141552     2.000000
Name: purchase_count, dtype: float64

## 6. Collaborative Filtering Model



#### Using purchase count

In [84]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 2408 observations with 421 users and 668 items.

Data prepared in: 0.005287s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 5.405ms                             | 100              | 668             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.007074s

#### Using purchase dummy

In [85]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 2408 observations with 423 users and 654 items.

Data prepared in: 0.00532s

...

| 65.096ms                            | 100              | 654             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.066835s

#### Using normalized purchase count

In [86]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 1477 observations with 403 users and 118 items.

Data prepared in: 0.004013s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 34.036ms                            | 100              | 118             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.035439s

#### Using purchase count

In [87]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 2408 observations with 421 users and 668 items.

Data prepared in: 0.005358s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 3.831ms                             | 100              | 668             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.005364s

#### Using purchase dummy

In [88]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 2408 observations with 423 users and 654 items.

Data prepared in: 0.004825s

...

| 3.923ms                             | 100              | 654             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.0056s

#### Using normalized purchase count

In [89]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 1477 observations with 403 users and 118 items.

Data prepared in: 0.004164s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 32.594ms                            | 100              | 118             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.03409s

In [90]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Models on purchase counts

In [91]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts

Precision and recall summary statistics by cutoff
+--------+-----------------+------------------+
| cutoff |  mean_precision |   mean_recall    |
+--------+-----------------+------------------+
|   1    |       0.0       |       0.0        |
|   2    | 0.0036496350365 | 0.00304136253041 |
|   3    | 0.0377128953771 | 0.0726364268335  |
|   4    | 0.0401459854015 |  0.102137643379  |
|   5    | 0.0423357664234 |  0.140154675009  |
|   6    |  0.036496350365 |  0.141675356274  |
|   7    | 0.0318039624609 |  0.142891901286  |
|   8    | 0.0287408759124 |  0.150191171359  |
|   9    | 0.0304136253041 |  0.184558567953  |
|   10   | 0.0288321167883 |  0.194899200556  |
+--------+-----------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 1.60500607518

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|  4298560   |   1   | 0.0  |
+------------+-------+

#### Models on purchase dummy

In [92]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy

Precision and recall summary statistics by cutoff
+--------+-------------------+------------------+
| cutoff |   mean_precision  |   mean_recall    |
+--------+-------------------+------------------+
|   1    |        0.0        |       0.0        |
|   2    |        0.0        |       0.0        |
|   3    |        0.0        |       0.0        |
|   4    |        0.0        |       0.0        |
|   5    |        0.0        |       0.0        |
|   6    | 0.000599520383693 | 0.00089928057554 |
|   7    |  0.00154162384378 | 0.00509592326139 |
|   8    |  0.00134892086331 | 0.00509592326139 |
|   9    |  0.00119904076739 | 0.00509592326139 |
|   10   |  0.00107913669065 | 0.00509592326139 |
+--------+-------------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|  2971920   |   2   | 0.0  |
+----

#### Models on normalized purchase frequency

In [93]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts

Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    | 0.0046511627907  | 0.00116279069767 |
|   2    | 0.0116279069767  | 0.0139534883721  |
|   3    | 0.0124031007752  | 0.0193798449612  |
|   4    | 0.0093023255814  | 0.0193798449612  |
|   5    | 0.00744186046512 | 0.0193798449612  |
|   6    | 0.0062015503876  | 0.0193798449612  |
|   7    | 0.00598006644518 | 0.0240310077519  |
|   8    | 0.00523255813953 | 0.0240310077519  |
|   9    | 0.0046511627907  | 0.0240310077519  |
|   10   | 0.00418604651163 | 0.0240310077519  |
+--------+------------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 0.378819012548

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|  4109728   |   1   | 0.0  |


## 8. Final Output
* In this step, we would like to manipulate format for recommendation output to one we can export to csv, and also a function that will return recommendation list given a customer ID.
* We need to first rerun the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

In [94]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 3010 observations with 436 users and 746 items.

Data prepared in: 0.006035s

...

| 37.66ms                             | 100              | 746             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.039598s

### 8.1. CSV output file

In [95]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(4360, 4)


,customerId,productId,score,rank
0,2698080,1688064,0.141521,1
1,2698080,1684816,0.120814,2
2,2698080,1683248,0.112441,3
3,2698080,1650992,0.108754,4
4,2698080,1685040,0.102552,5


In [96]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define a function to create a desired output

In [97]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('../output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [98]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(436, 1)


,recommendedProducts
customerId,
1121904,1698032|1570912|1120672|1901872|1125488|168291...
1122128,1698032|1120672|1570912|1649872|1902320|112022...
1122352,1698032|1570912|1132096|1120672|1649872|229196...
1133888,1698032|1570912|1931776|1120672|1121008|235692...
1134224,1698032|1120672|1649872|1121008|1132096|229196...


### 8.2. Customer recommendation function

In [99]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

Customer not found.


4

In [104]:
customer_recomendation(2698080)

recommendedProducts    1698032|1570912|1120672|1901872|1125488|168761...
Name: 2698080, dtype: object

In [102]:
customer_recomendation(1)

Customer not found.


1